# accuracy と r2_score の比較

現状の Evaluator モジュールと、テストデータをベースに、結果比較を行います。

## (1) テストデータ／環境準備

In [1]:
'''
    プロトタイピング用のパスと、Botライブラリーパスを取得／設定します
'''
import sys
import os

prototype_dir = os.path.join(os.getcwd(), '..')
prototype_dir = os.path.abspath(prototype_dir)

learning_dir = os.path.join(prototype_dir, '..')
learning_dir = os.path.abspath(learning_dir)
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

print('prototype_dir=%s\nlearning_dir=%s' % (prototype_dir, learning_dir))

prototype_dir=/Users/makmorit/GitHub/donusagi-bot/learning/prototype
learning_dir=/Users/makmorit/GitHub/donusagi-bot/learning


In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定したものを使用します。
'''
csv_file_name = 'test_daikin_conversation.csv'
original_csv_dir = os.path.join(learning_dir, 'learning/tests/engine/fixtures/')
original_file_path = os.path.join(original_csv_dir, csv_file_name)

csv_dir = os.path.join(prototype_dir, 'resources')

import shutil
shutil.copy2(original_file_path, csv_dir)
copied_csv_file_path = os.path.join(csv_dir, csv_file_name)

print('CSV file for test=[%s]' % copied_csv_file_path)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]


## (2) 既存モジュールをカスタマイズ

In [3]:
'''
    Bot/Reply モジュールをカスタマイズした
    BotForLocalTest/ReplyForLocalTest モジュールは、
    {prototype_dir}/modules 配下に格納されています
    （ローカル環境から MySQLdb/dataset に接続できないための措置）
'''
from prototype.modules.BotForLocalTest import Bot
from prototype.modules.ReplyForLocalTest import Reply
from learning.core.learn.learning_parameter import LearningParameter

'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)
csv_file_path = copied_csv_file_path
csv_file_encoding = 'utf-8'

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## (3) 学習＋ accuracy による評価実行

In [4]:
'''
    Bot クラスを生成し学習実行--->交差検証の実行

    引数 scoring を指定できるようにしてあります
'''
bot_id_accr = 9990
bot_accr = Bot(bot_id_accr, learning_parameter)
evaluator_acc = bot_accr.learn(csv_file_path=csv_file_path, 
                      csv_file_encoding=csv_file_encoding,
                      scoring=None) # <--- 既存の __accuracy_score を使用します
evaluator_acc.accuracy

learning_parameter: {'_params_for_algorithm': {}, '_algorithm': 0, '_include_failed_data': False, '_classify_threshold': None, '_include_tag_vector': False}
2017/03/10 PM 07:53:50 learning_parameter: {'_params_for_algorithm': {}, '_algorithm': 0, '_include_failed_data': False, '_classify_threshold': None, '_include_tag_vector': False}
start Bot#learn
2017/03/10 PM 07:53:51 start Bot#learn
TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/03/10 PM 07:53:51 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
TextArray#__init__ start
2017/03/10 PM 07:53:51 TextArray#__init__ start
TextArray#to_vec start
2017/03/10 PM 07:53:51 TextArray#to_vec start
TextArray#to_vec end
2017/03/10 PM 07:54:11 TextArray#to_vec end
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
20

0.98417794083925703

## (4) 学習＋ r2_score による評価実行

In [5]:
'''
    Bot クラスを生成し学習実行--->交差検証の実行

    引数 scoring をカスタマイズできるようにしてあります
'''
bot_id_r2 = 9991
bot_r2 = Bot(bot_id_r2, learning_parameter)
evaluator_r2 = bot_r2.learn(csv_file_path=csv_file_path, 
                      csv_file_encoding=csv_file_encoding,
                      scoring='r2') # <--- r2_score を使用します
evaluator_r2.accuracy

learning_parameter: {'_params_for_algorithm': {}, '_algorithm': 0, '_include_failed_data': False, '_classify_threshold': None, '_include_tag_vector': False}
2017/03/10 PM 07:57:57 learning_parameter: {'_params_for_algorithm': {}, '_algorithm': 0, '_include_failed_data': False, '_classify_threshold': None, '_include_tag_vector': False}
start Bot#learn
2017/03/10 PM 07:57:57 start Bot#learn
TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/03/10 PM 07:57:57 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
TextArray#__init__ start
2017/03/10 PM 07:57:57 TextArray#__init__ start
TextArray#to_vec start
2017/03/10 PM 07:57:57 TextArray#to_vec start
TextArray#to_vec end
2017/03/10 PM 07:58:18 TextArray#to_vec end
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
20

0.96553822690252789

## (5) 結果比較

同じ訓練データを用いて学習した結果、

- accuracy による評価＝0.98417794083925703


- r2_score による評価＝0.96553822690252789

となりました。

いずれも１に近づくほど良い予想モデルという意味合いであり、大差はないかと存じます。

ただし、accuracy が正解率なのに対し、r2_score はサンプルと予測値との乖離を示す指標であるところが異なります（後述）。

### 補足：R2（決定係数）について

基本的には、データに対するモデルの「当てはまり具合」を示す指標とのことです。

正解率と比べると、体感的にわかりづらい指標かもしれません。


- 決定係数に関する概説（Wikipedia）

 https://ja.wikipedia.org/wiki/決定係数
 
 引用ですが・・・

 R2 = 1 - [{(サンプル値 - 予測値)の２乗}の総和] / [{(サンプル値 - 全サンプル平均値)の２乗}の総和]

 モデルの予測値が、サンプルと乖離していなければ、相対残差（１からマイナスされている右辺値）は限りなく小さくなるはずです。
 
 したがいまして、R2 の値が１に近ければ近いほど、訓練データに対して予測モデルがぴったりとフィットしていることになります。

### ご参考：r2_score 関数 の実装

下記は、R2 のスコアを計算している部分を、ソースコードから抜粋したものです。

おおむね、Wikipedia での説明通り実装されているようです。


- scikit-learn における r2_score 関数の仕様

 http://scikit-learn.org/0.17/modules/generated/sklearn.metrics.r2_score.html#sklearn-metrics-r2-score

In [9]:
def r2_score(y_true, y_pred,
             sample_weight=None,
             multioutput=None):
    #（中略）
    numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
                                                      dtype=np.float64)
    denominator = (weight * (y_true - np.average(
        y_true, axis=0, weights=sample_weight)) ** 2).sum(axis=0,
                                                          dtype=np.float64)
    nonzero_denominator = denominator != 0
    nonzero_numerator = numerator != 0
    valid_score = nonzero_denominator & nonzero_numerator
    output_scores = np.ones([y_true.shape[1]])
    output_scores[valid_score] = 1 - (numerator[valid_score] /
                                      denominator[valid_score])
    #（以下略）
    return np.average(output_scores, weights=avg_weights)